# Analysis on cleaned dataset from Data.ipynb

In [ ]:
#only perform once in the beginning
#%pip install linearmodels

In [221]:
import pandas as pd
import numpy as np

# Fixed effects regression:
from linearmodels import PanelOLS
import statsmodels.api as sm


In [222]:
# import the csv data from Data.ipynb
data = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/data_clean.csv')
houshold = pd.read_csv('/Users/maxweber/Desktop/DataMasterThesis/houshold_clean.csv', index_col=['SSUID', 'month_total'])

# Analysis base modification
- with & without groupings to make sure the definition is correct
- with my dataset and with POWELL dataset
- with time and entity effect


In [241]:
houshold.head()

MultiIndex([( 19128000276,  6.0),
            ( 19128000276,  7.0),
            ( 19128000276,  8.0),
            ( 19128000276,  9.0),
            ( 19128000276, 10.0),
            ( 19128000276, 11.0),
            ( 19128000276, 12.0),
            ( 19128000276, 13.0),
            ( 19128000932,  6.0),
            ( 19128000932,  7.0),
            ...
            (955925986857, 13.0),
            (955925986857, 14.0),
            (955958398688,  7.0),
            (955958398688,  8.0),
            (955958398688,  9.0),
            (955958398688, 10.0),
            (955958398688, 11.0),
            (955958398688, 12.0),
            (955958398688, 13.0),
            (955958398688, 14.0)],
           names=['SSUID', 'month_total'], length=209904)

In [223]:
# with dummy * amount:
exog_vars = houshold[['erebate-2', 'erebate-1', 'erebate0', 'erebate1','erebate2', 'erebate3', 'erebate4', 'erebate5', 'erebate6', 
                      'erebate7', 'erebate8', 'erebate9', 'erebate10', 'erebate11']].multiply(houshold['ERBATAMT'], axis = 'index')
#exog_vars['month_total'] = pd.Categorical(houshold['month_total'])
exog = sm.add_constant(exog_vars)

model = PanelOLS(houshold['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()

Dep. Variable:,tfearn,R-squared:,8.375e-05
Estimator:,PanelOLS,R-squared (Between):,-0.0007
No. Observations:,209904,R-squared (Within):,0.0002
Date:,"Fri, Jun 03 2022",R-squared (Overall):,-0.0006
Time:,13:19:41,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.0987
Entities:,26238,P-value,0.3526
Avg Obs:,8.0000,Distribution:,"F(14,183642)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.0987


In [225]:
# die obere regression nochmal machen aber rebate rebate0+rebate1 und rebate2+rebate3 gruppieren: 
# addieren der dummies, die rows bleiben aber so bestehen und werden nicht aggregiert

powell_hh = houshold[['ERBATAMT', 'tfearn']]
powell_hh['reb'] = houshold['erebate-1'] + houshold['erebate0']
powell_hh['reb_lag'] = houshold['erebate1'] + houshold['erebate1']
powell_hh['reb_lag1'] = houshold['erebate4'] + houshold['erebate3']
powell_hh['reb_lag2'] = houshold['erebate5'] + houshold['erebate6'] + houshold['erebate7'] + houshold['erebate8']

exog_vars = powell_hh[['reb', 'reb_lag']].multiply(powell_hh['ERBATAMT'], axis = 'index') #, 'reb_lag1', 'reb_lag2'

exog = sm.add_constant(exog_vars)

model = PanelOLS(powell_hh['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()


# include interaction terms? 
# what is the 2SLS approach in Powell? is the first stage Rebate = gamma (interaction terms of HH characteristica)?
# read the stata code again (especially concerning the interacgtion term!)
# set tfearn = 0 if tfearn < 0.

# don't use houshold dataframe that is grouped but use the regular one with pp as index and cluster by hh in the regressions.

<ipython-input-225-61c28399f0d0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb'] = houshold['erebate-1'] + houshold['erebate0']
<ipython-input-225-61c28399f0d0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb_lag'] = houshold['erebate1'] + houshold['erebate1']
<ipython-input-225-61c28399f0d0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

Dep. Variable:,tfearn,R-squared:,1.347e-05
Estimator:,PanelOLS,R-squared (Between):,6.237e-05
No. Observations:,209904,R-squared (Within):,-3.647e-05
Date:,"Fri, Jun 03 2022",R-squared (Overall):,5.056e-05
Time:,13:20:33,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.2373
Entities:,26238,P-value,0.2902
Avg Obs:,8.0000,Distribution:,"F(2,183654)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.2373


In [227]:
# with dummy only:
exog_vars = houshold[['erebate-2', 'erebate-1', 'erebate0', 'erebate1','erebate2', 'erebate3', 'erebate4', 'erebate5', 'erebate6', 
                      'erebate7', 'erebate8', 'erebate9', 'erebate10', 'erebate11']]

exog = sm.add_constant(exog_vars)

model = PanelOLS(houshold['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()


# there is a cluster option as in the stata code

Dep. Variable:,tfearn,R-squared:,0.0001
Estimator:,PanelOLS,R-squared (Between):,0.0005
No. Observations:,209904,R-squared (Within):,-0.0001
Date:,"Fri, Jun 03 2022",R-squared (Overall):,0.0004
Time:,13:21:26,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.5660
Entities:,26238,P-value,0.0802
Avg Obs:,8.0000,Distribution:,"F(14,183642)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,1.5660


In [228]:
# next step: estimation with grouping:
totinc_dict = {
    1: range(0,2499),
    2: range(2500,5999),
    3: range(5000,12499),
    4: range(12500,99999)
}

reb_dummies_lst = ['erebate-1', 'erebate0', 'erebate1','erebate2', 'erebate3', 'erebate4']
totinc_dict.keys()

dict_keys([1, 2, 3, 4])

In [229]:
# alt:
def get_grouping(df_column, group_dict, reb_dummies_df):
    # get flag indicating the group with the dictionary-mapping:
    new_col = df_column.apply(lambda x: next((k for k, v in group_dict.items() if x in v), 0))
    new_col = new_col.rename(new_col.name + '_flag')
    new_cols = pd.get_dummies(new_col) # get dummies from group flag


   # create interaction dummies (group-dummy * rebatelag_dummy) iterating over each group_id and concatenating in the end:
    df_group_interactions = pd.DataFrame(df_column) # for initialization with correct index
    group_lst = group_dict.keys()
    for group_id in group_lst:
        dummies_group = reb_dummies_df.multiply(new_cols[group_id], axis = 'index')
        
        new_dummy_names = []
        for name in dummies_group.columns:
            new_dummy_names.append(name + '-j' + str(group_id))
        dummies_group.columns = new_dummy_names
        df_group_interactions = df_group_interactions.merge(dummies_group,left_index= True, right_index=True)
        #df_group_dummies.replace(np.NaN, 0, inplace = True)
    return df_group_interactions


get_grouping(houshold['TFTOTINC'], totinc_dict, houshold[reb_dummies_lst])

TFTOTINC  erebate-1-j1  erebate0-j1  erebate1-j1  \
SSUID        month_total                                                     
19128000276  6.0              1099           0.0          0.0          0.0   
             7.0              1099           0.0          0.0          0.0   
             8.0              1099           0.0          0.0          0.0   
             9.0              1099           0.0          0.0          0.0   
             10.0              381           0.0          0.0          0.0   
...                            ...           ...          ...          ...   
955958398688 10.0            10104           0.0          0.0          0.0   
             11.0            10333           0.0          0.0          0.0   
             12.0            10333           0.0          0.0          0.0   
             13.0            10333           0.0          0.0          0.0   
             14.0            10333           0.0          0.0          0.0   

                          erebate2-j1  erebate3-j1  erebate4-j1  erebate-1-j2  \
SSUID        month_total                                                        
19128000276  6.0                  0.0          0.0          0.0           0.0   
             7.0                  0.0          0.0          0.0           0.0   
             8.0                  0.0          0.0          0.0           0.0   
             9.0                  0.0          0.0          0.0           0.0   
             10.0                 0.0          0.0          0.0           0.0   
...                               ...          ...          ...           ...   
955958398688 10.0                 0.0          0.0          0.0           0.0   
             11.0                 0.0          0.0          0.0           0.0   
             12.0                 0.0          0.0          0.0           0.0   
             13.0                 0.0          0.0          0.0           0.0   
             14.0                 0.0          0.0          0.0           0.0   

                          erebate0-j2  erebate1-j2  ...  erebate1-j3  \
SSUID        month_total                            ...                
19128000276  6.0                  0.0          0.0  ...          0.0   
             7.0                  0.0          0.0  ...          0.0   
             8.0                  0.0          0.0  ...          0.0   
             9.0                  0.0          0.0  ...          0.0   
             10.0                 0.0          0.0  ...          0.0   
...                               ...          ...  ...          ...   
955958398688 10.0                 0.0          0.0  ...          1.0   
             11.0                 0.0          0.0  ...          0.0   
             12.0                 0.0          0.0  ...          0.0   
             13.0                 0.0          0.0  ...          0.0   
             14.0                 0.0          0.0  ...          0.0   

                          erebate2-j3  erebate3-j3  erebate4-j3  erebate-1-j4  \
SSUID        month_total                                                        
19128000276  6.0                  0.0          0.0          0.0           0.0   
             7.0                  0.0          0.0          0.0           0.0   
             8.0                  0.0          0.0          0.0           0.0   
             9.0                  0.0          0.0          0.0           0.0   
             10.0                 0.0          0.0          0.0           0.0   
...                               ...          ...          ...           ...   
955958398688 10.0                 0.0          0.0          0.0           0.0   
             11.0                 0.0          1.0          0.0           0.0   
             12.0                 0.0          0.0          1.0           0.0   
             13.0                 0.0          0.0          0.0           0.0   
             14.0                 0.0          0

In [230]:
test_group_dummies = get_grouping(houshold['TFTOTINC'], totinc_dict, houshold[reb_dummies_lst])

print(test_group_dummies.shape)
print(test_group_dummies['erebate3-j3'].sum())
reb_interaction_dummies = test_group_dummies.columns
reb_interaction_dummies

(209904, 25)
5534.0


Index(['TFTOTINC', 'erebate-1-j1', 'erebate0-j1', 'erebate1-j1', 'erebate2-j1',
       'erebate3-j1', 'erebate4-j1', 'erebate-1-j2', 'erebate0-j2',
       'erebate1-j2', 'erebate2-j2', 'erebate3-j2', 'erebate4-j2',
       'erebate-1-j3', 'erebate0-j3', 'erebate1-j3', 'erebate2-j3',
       'erebate3-j3', 'erebate4-j3', 'erebate-1-j4', 'erebate0-j4',
       'erebate1-j4', 'erebate2-j4', 'erebate3-j4', 'erebate4-j4'],
      dtype='object')

In [232]:
df_w_group_interaction = test_group_dummies.merge(houshold[['tfearn', 'ERBATAMT']], left_index=True, right_index=True)
#df_w_group_interaction['month_total'] = pd.Categorical(houshold['month_total'])
df_w_group_interaction

TFTOTINC  erebate-1-j1  erebate0-j1  erebate1-j1  \
SSUID        month_total                                                     
19128000276  6.0              1099           0.0          0.0          0.0   
             7.0              1099           0.0          0.0          0.0   
             8.0              1099           0.0          0.0          0.0   
             9.0              1099           0.0          0.0          0.0   
             10.0              381           0.0          0.0          0.0   
...                            ...           ...          ...          ...   
955958398688 10.0            10104           0.0          0.0          0.0   
             11.0            10333           0.0          0.0          0.0   
             12.0            10333           0.0          0.0          0.0   
             13.0            10333           0.0          0.0          0.0   
             14.0            10333           0.0          0.0          0.0   

                          erebate2-j1  erebate3-j1  erebate4-j1  erebate-1-j2  \
SSUID        month_total                                                        
19128000276  6.0                  0.0          0.0          0.0           0.0   
             7.0                  0.0          0.0          0.0           0.0   
             8.0                  0.0          0.0          0.0           0.0   
             9.0                  0.0          0.0          0.0           0.0   
             10.0                 0.0          0.0          0.0           0.0   
...                               ...          ...          ...           ...   
955958398688 10.0                 0.0          0.0          0.0           0.0   
             11.0                 0.0          0.0          0.0           0.0   
             12.0                 0.0          0.0          0.0           0.0   
             13.0                 0.0          0.0          0.0           0.0   
             14.0                 0.0          0.0          0.0           0.0   

                          erebate0-j2  erebate1-j2  ...  erebate3-j3  \
SSUID        month_total                            ...                
19128000276  6.0                  0.0          0.0  ...          0.0   
             7.0                  0.0          0.0  ...          0.0   
             8.0                  0.0          0.0  ...          0.0   
             9.0                  0.0          0.0  ...          0.0   
             10.0                 0.0          0.0  ...          0.0   
...                               ...          ...  ...          ...   
955958398688 10.0                 0.0          0.0  ...          0.0   
             11.0                 0.0          0.0  ...          1.0   
             12.0                 0.0          0.0  ...          0.0   
             13.0                 0.0          0.0  ...          0.0   
             14.0                 0.0          0.0  ...          0.0   

                          erebate4-j3  erebate-1-j4  erebate0-j4  erebate1-j4  \
SSUID        month_total                                                        
19128000276  6.0                  0.0           0.0          0.0          0.0   
             7.0                  0.0           0.0          0.0          0.0   
             8.0                  0.0           0.0          0.0          0.0   
             9.0                  0.0           0.0          0.0          0.0   
             10.0                 0.0           0.0          0.0          0.0   
...                               ...           ...          ...          ...   
955958398688 10.0                 0.0           0.0          0.0          0.0   
             11.0                 0.0           0.0          0.0          0.0   
             12.0                 1.0           0.0          0.0          0.0   
             13.0                 0.0           0.0          0.0          0.0   
             14.0                 0.0           

In [233]:
# with interaction dummies for total income groups:
exog_vars = df_w_group_interaction[['erebate-1-j1', 'erebate0-j1', 'erebate1-j1', 'erebate2-j1',
                                    'erebate3-j1', 'erebate4-j1', 'erebate-1-j2', 'erebate0-j2',
                                    'erebate1-j2', 'erebate2-j2', 'erebate3-j2', 'erebate4-j2',
                                   'erebate-1-j3', 'erebate0-j3', 'erebate1-j3', 'erebate2-j3',
                                   'erebate3-j3', 'erebate4-j3', 'erebate-1-j4', 'erebate0-j4',
                                   'erebate1-j4', 'erebate2-j4', 'erebate3-j4', 'erebate4-j4']]#.multiply(df_w_group_interaction['ERBATAMT'], axis = 'index')
#exog_vars['month_total'] = pd.Categorical(houshold['month_total'])

exog = sm.add_constant(exog_vars)

model = PanelOLS(houshold['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)

model.fit()

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,tfearn,R-squared:,0.0476
Estimator:,PanelOLS,R-squared (Between):,0.0786
No. Observations:,209904,R-squared (Within):,0.0473
Date:,"Fri, Jun 03 2022",R-squared (Overall):,0.0749
Time:,13:22:18,Log-likelihood,-1.886e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,382.11
Entities:,26238,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(24,183632)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,382.11


# 2SLS approach by hand

In [242]:
# analysis on subset with interactions terms:
powell_hh = houshold[['ERBATAMT', 'tfearn']]
powell_hh['reb'] = houshold['erebate0'] + houshold['erebate1']
powell_hh['reb_lag'] = houshold['erebate3'] + houshold['erebate3']
powell_hh['erbatamt'] = powell_hh['ERBATAMT'].multiply(powell_hh['reb'], axis = 'index')  # so that it is 0 in the months without rebate payout
powell_hh['erbatamt_lag'] = powell_hh['ERBATAMT'].multiply(powell_hh['reb_lag'], axis = 'index')
#interaction terms based on  marital status, household size, month (and month relative to interview month)
powell_hh[['ems', 'EHHNUMPP']] = houshold[['ems', 'EHHNUMPP']]

ems_interactions = powell_hh[['reb', 'reb_lag']].multiply(powell_hh['ems'], axis='index')
ems_interactions.columns = ['ems_reb', 'ems_reb_lag']

ehhnumpp_dict = {1:range(1,2), 2:range(2,3), 3: range(3,4), 4: range(5,20)} # group 4 has to be omitted to get full rank
hhsize_interactions = get_grouping(powell_hh['EHHNUMPP'], ehhnumpp_dict, powell_hh[['reb', 'reb_lag']])
powell_hh = pd.concat([powell_hh,ems_interactions,hhsize_interactions], axis = 1)
powell_hh.columns

<ipython-input-242-212839042b47>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb'] = houshold['erebate0'] + houshold['erebate1']
<ipython-input-242-212839042b47>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powell_hh['reb_lag'] = houshold['erebate3'] + houshold['erebate3']
<ipython-input-242-212839042b47>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

Index(['ERBATAMT', 'tfearn', 'reb', 'reb_lag', 'erbatamt', 'erbatamt_lag',
       'ems', 'EHHNUMPP', 'ems_reb', 'ems_reb_lag', 'EHHNUMPP', 'reb-j1',
       'reb_lag-j1', 'reb-j2', 'reb_lag-j2', 'reb-j3', 'reb_lag-j3', 'reb-j4',
       'reb_lag-j4'],
      dtype='object')

### strategy

- get fitted values for reb_amount and reb_amount_lag
- plug into second stage

In [243]:
# rebate amount with reb and reb lag only 
exog_vars = powell_hh[['reb', 'reb_lag']]
exog = sm.add_constant(exog_vars)

model_base_reb = PanelOLS(powell_hh['erbatamt'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_base_reb = model_base_reb.fit()
params_base_reb.params

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


const        0.403550
reb        973.864794
reb_lag      0.058667
Name: parameter, dtype: float64

In [244]:
# rebate amount with reb and reb lag only 
exog_vars = powell_hh[['reb', 'reb_lag']]
exog = sm.add_constant(exog_vars)

model_base_reb_lag = PanelOLS(powell_hh['erbatamt_lag'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_base_reb_lag = model_base_reb_lag.fit()
params_base_reb_lag.params

const        0.695966
reb         -3.633992
reb_lag    983.821872
Name: parameter, dtype: float64

In [245]:
# first stage fitted rebate with interactions 
exog_vars = powell_hh[['reb', 'reb_lag', 'reb-j1', 'reb_lag-j1', 'reb-j2', 'reb_lag-j2', 'reb-j3', 'reb_lag-j3', 'ems_reb', 'ems_reb_lag']]
exog = sm.add_constant(exog_vars)
exog
model_interactions = PanelOLS(powell_hh['ERBATAMT'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_interactions = model_interactions.fit()
params_interactions.params

const          725.484835
reb             96.992565
reb_lag         42.640645
reb-j1         -39.578295
reb_lag-j1     -25.292527
reb-j2         -34.182148
reb_lag-j2     -20.943055
reb-j3         -19.242875
reb_lag-j3     -14.250386
ems_reb         10.852697
ems_reb_lag      2.245350
Name: parameter, dtype: float64

In [260]:
# generate fitted values for the second stage regression:
print(params_base_reb.params)
erbatamt_fitted_base_reb = params_base_reb.fitted_values
erbatamt_fitted_base_reb.columns = ['erbatamt_fitted_base_reb']

print(params_base_reb_lag.params)
erbatamt_fitted_base_reb_lag = params_base_reb_lag.fitted_values
erbatamt_fitted_base_reb_lag.columns = ['erbatamt_fitted_base_reb_lag']

powell_hh = pd.concat([powell_hh,erbatamt_fitted_base_reb, erbatamt_fitted_base_reb_lag], axis = 1)
powell_hh

const        0.403550
reb        973.864794
reb_lag      0.058667
Name: parameter, dtype: float64
const        0.695966
reb         -3.633992
reb_lag    983.821872
Name: parameter, dtype: float64


ERBATAMT  tfearn  reb  reb_lag  erbatamt  \
SSUID        month_total                                             
19128000276  6.0               0.0       0  0.0      0.0       0.0   
             7.0               0.0       0  0.0      0.0       0.0   
             8.0               0.0       0  0.0      0.0       0.0   
             9.0               0.0       0  0.0      0.0       0.0   
             10.0              0.0       0  0.0      0.0       0.0   
...                            ...     ...  ...      ...       ...   
955958398688 10.0           2000.0   10000  1.0      0.0    2000.0   
             11.0           1800.0   10000  0.0      2.0       0.0   
             12.0           1800.0   10000  0.0      0.0       0.0   
             13.0           1800.0   10000  0.0      0.0       0.0   
             14.0           1800.0   10000  0.0      0.0       0.0   

                          erbatamt_lag  ems  EHHNUMPP  EHHNUMPP  ems_reb  ...  \
SSUID        month_total                                                  ...   
19128000276  6.0                   0.0    1         3         3      0.0  ...   
             7.0                   0.0    1         3         3      0.0  ...   
             8.0                   0.0    1         3         3      0.0  ...   
             9.0                   0.0    1         3         3      0.0  ...   
             10.0                  0.0    1         3         3      0.0  ...   
...                                ...  ...       ...       ...      ...  ...   
955958398688 10.0                  0.0    1         5         5      1.0  ...   
             11.0               3600.0    1         5         5      0.0  ...   
             12.0                  0.0    1         5         5      0.0  ...   
             13.0                  0.0    1         5         5      0.0  ...   
             14.0                  0.0    1         5         5      0.0  ...   

                          reb-j1  reb_lag-j1  reb-j2  reb_lag-j2  reb-j3  \
SSUID        month_total                                                   
19128000276  6.0             0.0         0.0     0.0         0.0     0.0   
             7.0             0.0         0.0     0.0         0.0     0.0   
             8.0             0.0         0.0     0.0         0.0     0.0   
             9.0             0.0         0.0     0.0         0.0     0.0   
             10.0            0.0         0.0     0.0         0.0     0.0   
...                          ...         ...     ...         ...     ...   
955958398688 10.0            0.0         0.0     0.0         0.0     0.0   
             11.0            0.0         0.0     0.0         0.0     0.0   
             12.0            0.0         0.0     0.0         0.0     0.0   
             13.0            0.0         0.0     0.0         0.0     0.0   
             14.0            0.0         0.0     0.0         0.0     0.0   

                          reb_lag-j3  reb-j4  reb_lag-j4  \
SSUID        month_total                                   
19128000276  6.0                 0.0     0.0         0.0   
             7.0                 0.0     0.0         0.0   
             8.0                 0.0     0.0         0.0   
             9.0                 0.0     0.0         0.0   
             10.0                0.0     0.0         0.0   
...                              ...     ...         ...   
955958398688 10.0                0.0     1.0         0.0   
             11.0                0.0     0.0         2.0   
             12.0                0.0     0.0         0.0   
             13.0                0.0     0.0         0.0   
             14.0                0.0     0.0         0.0   

                          erbatamt_fitted_base_reb  \
SSUID        month_total                             
19128000276  6.0                          0.403550   
             7.0                          0.403550   
             8.0                          0.403550   
             9.0     

## Second Stage

In [247]:
# no interaction terms, using fitted value from 1st stage without interaction terms
exog_vars = powell_hh[['erbatamt_fitted_base_reb', 'erbatamt_fitted_base_reb_lag']]
exog = sm.add_constant(exog_vars)
exog
model_second_stage = PanelOLS(powell_hh['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_second_stage = model_second_stage.fit()
params_second_stage

/Users/maxweber/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Dep. Variable:,tfearn,R-squared:,5.459e-06
Estimator:,PanelOLS,R-squared (Between):,-4.339e-07
No. Observations:,209904,R-squared (Within):,-3.07e-05
Date:,"Sun, Jun 05 2022",R-squared (Overall):,-4.051e-06
Time:,14:38:50,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.5013
Entities:,26238,P-value,0.6057
Avg Obs:,8.0000,Distribution:,"F(2,183654)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,0.5013


In [248]:
# with interactin termns, using fitted value from 1st stage without interaction terms:

exog_vars = powell_hh[['erbatamt_fitted_base_reb', 'erbatamt_fitted_base_reb_lag', 
                       'reb-j1', 'reb_lag-j1', 'reb-j2', 'reb_lag-j2', 'reb-j3', 'reb_lag-j3', 'ems_reb', 'ems_reb_lag']]
exog = sm.add_constant(exog_vars)
exog
model_second_stage_interactions = PanelOLS(powell_hh['tfearn'], exog, entity_effects = True, time_effects = True, check_rank = True, drop_absorbed=True)
params_second_stage_interactions = model_second_stage_interactions.fit()
params_second_stage_interactions

# do I need to multiply the interaction terms with the amount?

Dep. Variable:,tfearn,R-squared:,4.735e-05
Estimator:,PanelOLS,R-squared (Between):,0.0004
No. Observations:,209904,R-squared (Within):,9.271e-06
Date:,"Sun, Jun 05 2022",R-squared (Overall):,0.0004
Time:,14:39:24,Log-likelihood,-1.891e+06
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.8697
Entities:,26238,P-value,0.5611
Avg Obs:,8.0000,Distribution:,"F(10,183646)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,0.8697
